In [10]:
import nltk
from nltk.corpus import brown, stopwords
from nltk import RegexpTokenizer, FreqDist, ngrams
import numpy as np
import pandas as pd
import re, pickle, random, math

In [21]:
def make_words_grams():
# print(list(brown.words()[:500]))
    words = [w.lower() for w in brown.words()]
    # words = [w for w in words if w not in stopwords.words('english')]
    estopwords = set(stopwords.words('english'))
    words = [w for w in words if w not in estopwords]
    rem = re.compile(r'[a-z][a-z\']*')
    words = [w for w in words if rem.search(w)]
    
    freq_words = FreqDist(words).most_common(20000)
    freq_words = set([w for (w,f) in freq_words])
    words = [w for w in words if w in freq_words]
    
    grams = list(ngrams(words, 15))
    pickle.dump(words, open("words.txt", "wb"))
    pickle.dump(grams, open("grams.txt", "wb"))
    return words, grams

In [14]:
## load words and grams
with open("words.txt", "rb") as fp:
    words = pickle.load(fp)
with open("grams.txt", "rb") as fp:
    grams = pickle.load(fp)

In [23]:
words, grams = make_words_grams()

In [24]:
vocab = list(set(words))
voc_len = len(vocab)
matrix = pd.DataFrame(0, vocab, vocab, dtype=np.float32)
# print(matrix)

In [27]:
print(voc_len, matrix.shape, len(grams))


20000 (20000, 20000) 497712


In [28]:
for g in grams:
    mid_word = g[7]
#     print(mid_word)
    row = matrix.loc[mid_word]
    row.at[g[0]]  += 1
    row.at[g[1]]  += 1
    row.at[g[2]]  += 1
    row.at[g[3]]  += 1
    row.at[g[4]]  += 1
    row.at[g[5]]  += 1
    row.at[g[6]]  += 1
    row.at[g[8]]  += 1
    row.at[g[9]]  += 1
    row.at[g[10]] += 1
    row.at[g[11]] += 1
    row.at[g[12]] += 1
    row.at[g[13]] += 1
    row.at[g[14]] += 1

#     for i in range(len(g)):
#         if i == 7:
#             continue
#         row.at[g[i]] += 1
#         print(i, g[i], row.at[g[i]])


In [29]:
print(matrix.loc["play"].iloc[matrix.loc["play"].nonzero()])
# # matrix.loc[mid_word].nonzero()
# matrix.loc["atlanta's",'jury']
# matrix.iloc[""]

get             5.0
mankowski       1.0
football        1.0
good            8.0
believe         5.0
nowhere         1.0
keeps           1.0
cartoons        1.0
garson          1.0
learned         3.0
public          4.0
rising          1.0
earlier         1.0
called          7.0
equipment       1.0
maryland        1.0
dark            3.0
historian       1.0
buffalo         1.0
message         1.0
milstein        1.0
however         3.0
mahler          1.0
week            5.0
forth           1.0
emotion         1.0
charles         3.0
jacoby          1.0
sharply         1.0
suspiciously    1.0
               ... 
instructor      1.0
working         1.0
write           3.0
sidewalks       1.0
roger           1.0
latin           1.0
maintain        1.0
spectacular     3.0
haven't         2.0
affect          1.0
realism         2.0
building        1.0
humor           1.0
loss            2.0
wistfully       1.0
swing           1.0
charlie         1.0
destruction     1.0
department      1.0


In [20]:
matrix.shape

(20000, 20000)

In [ ]:
with open("1matrix" + str(random.random())[2:5] + ".txt", 'wb') as fp:
    pickle.dump(matrix[:20000], fp, protocol=4)
with open("2matrix" + str(random.random())[2:5] + ".txt", 'wb') as fp:
    pickle.dump(matrix[20000:], fp, protocol=4)

In [30]:
matrix.to_hdf("smatrix.hdf","mat1", mode='w')

# df.to_hdf('my_filename.hdf','mydata',mode='w')

# df = pd.read_hdf('my_filename.hdf','mydata')

In [ ]:
x =pd.DataFrame(0.0, ["a", "b", 'c'], ["A", "B", 'C'], dtype="int16")
print(x)
print(x.loc['a'])
print("index", x.index,"columns", x.columns)
for ind in x.index:
    print(ind)

In [3]:
matrix = pd.read_hdf('matrix.hdf','mat1')

In [4]:
cols = matrix.sum(axis=1)
rows = matrix.sum(axis=0)

In [5]:
# log_2 val / (row * col)
for r in matrix.index:
    row = matrix.loc[r]
    for c in matrix.columns:
        val = row.at[c]
        if val <= 0: 
            row.at[c] = 0
            continue
            
        ppmi = row.at[c] / (cols[c] * float(rows[r]))
        ppmi = max(0, math.log2(ppmi))
        row.at[c] = ppmi

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [nan] of <class 'float'>

In [6]:
matrix.columns

Index(['excommunicated',           'arco',     'condensing',           'catt',
               'heusen',  'unimpassioned',    'assyriology',   'fishing-boat',
             'hopkins'',        'gearing',
       ...
               'charts',      'exception',     'school-age',      'heffernan',
           'unofficial', 'russo-american',        'wastage',         'pulova',
                'fiend', 'oversimplified'],
      dtype='object', length=47915)